You may need to install it with  
$ easy_install beautifulsoup4

$ pip install beautifulsoup4

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
# import os
import requests

In [4]:
nyt = BeautifulSoup("https://www.nytimes.com/", 'html.parser')

C:\Users\halpe\.conda\envs\learn-env\lib\site-packages\bs4\__init__.py:357: UserWarning: "https://www.nytimes.com/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


In [4]:
nyt_get_request = requests.get("https://www.nytimes.com/", verify=False)

C:\Users\halpe\.conda\envs\learn-env\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [5]:
nyt = BeautifulSoup(nyt_get_request.content, 'html.parser')

From inspecting, all article titles are `h3`

In [6]:
nyt.h3

<h3 class="css-12g2h6t ee0hn7b0" color="primary" size="400">Signs of War Across Ukraine as Leader Vows to Fight<div class="css-13htjwu"></div></h3>

In [7]:
headlines = nyt.findAll('h3')
headlines[0:3]

[<h3 class="css-12g2h6t ee0hn7b0" color="primary" size="400">Signs of War Across Ukraine as Leader Vows to Fight<div class="css-13htjwu"></div></h3>,
 <h3 class="css-w4bgn6 ee0hn7b0" color="primary" size="300">U.S. to Impose Sanctions on Putin, Official Says</h3>,
 <h3 class="css-1dc6422 ee0hn7b0" color="primary" size="500">We’re tracking the Russian invasion of Ukraine.</h3>]

It's a bit of a mess when I just want the text. I can isolate it like this. 

In [8]:
headlines[0].text

'Signs of War Across Ukraine as Leader Vows to Fight'

In [9]:
all_headlines = [headline.text for headline in headlines]

In [10]:
all_headlines[0:4]

['Signs of War Across Ukraine as Leader Vows to Fight',
 'U.S. to Impose Sanctions on Putin, Official Says',
 'We’re tracking the Russian invasion of Ukraine.',
 'After Vicious Battle for Kharkiv, Wreckage and Artillery Booms']

# NLP Project

In [2]:
f = open('data_files/reut2-000.sgm', encoding='utf-8', errors='ignore')
dataFile = f.read()
dataFile[0:1000]

'<!DOCTYPE lewis SYSTEM "lewis.dtd">\n<REUTERS TOPICS="YES" LEWISSPLIT="TRAIN" CGISPLIT="TRAINING-SET" OLDID="5544" NEWID="1">\n<DATE>26-FEB-1987 15:01:01.79</DATE>\n<TOPICS><D>cocoa</D></TOPICS>\n<PLACES><D>el-salvador</D><D>usa</D><D>uruguay</D></PLACES>\n<PEOPLE></PEOPLE>\n<ORGS></ORGS>\n<EXCHANGES></EXCHANGES>\n<COMPANIES></COMPANIES>\n<UNKNOWN> \n&#5;&#5;&#5;C T\n&#22;&#22;&#1;f0704&#31;reute\nu f BC-BAHIA-COCOA-REVIEW   02-26 0105</UNKNOWN>\n<TEXT>&#2;\n<TITLE>BAHIA COCOA REVIEW</TITLE>\n<DATELINE>    SALVADOR, Feb 26 - </DATELINE><BODY>Showers continued throughout the week in\nthe Bahia cocoa zone, alleviating the drought since early\nJanuary and improving prospects for the coming temporao,\nalthough normal humidity levels have not been restored,\nComissaria Smith said in its weekly review.\n    The dry period means the temporao will be late this year.\n    Arrivals for the week ended February 22 were 155,221 bags\nof 60 kilos making a cumulative total for the season of 5.93\nml

In [3]:
soup = BeautifulSoup(dataFile, 'lxml')
content = soup.prettify()
print(content[0:300])

<!DOCTYPE lewis SYSTEM "lewis.dtd">
<html>
 <body>
  <reuters cgisplit="TRAINING-SET" lewissplit="TRAIN" newid="1" oldid="5544" topics="YES">
   <date>
    26-FEB-1987 15:01:01.79
   </date>
   <topics>
    <d>
     cocoa
    </d>
   </topics>
   <places>
    <d>
     el-salvador
    </d>
    <d>
  


In [4]:
topics = soup.find_all('topics')
type(topics)

bs4.element.ResultSet

In [5]:
topics = soup.find_all('topics')
topics[0:10]

[<topics><d>cocoa</d></topics>,
 <topics></topics>,
 <topics></topics>,
 <topics></topics>,
 <topics><d>grain</d><d>wheat</d><d>corn</d><d>barley</d><d>oat</d><d>sorghum</d></topics>,
 <topics><d>veg-oil</d><d>linseed</d><d>lin-oil</d><d>soy-oil</d><d>sun-oil</d><d>soybean</d><d>oilseed</d><d>corn</d><d>sunseed</d><d>grain</d><d>sorghum</d><d>wheat</d></topics>,
 <topics></topics>,
 <topics></topics>,
 <topics><d>earn</d></topics>,
 <topics><d>acq</d></topics>]

In [6]:
# instantiate an empty list
topic_list = list()

for x in topics:
    # turn bs4.tag into text and create a list of each article's topics
#     words = [i.text for i in x]
    words = [i.get_text() for i in x]
    #append this list to the larger list
    topic_list.append(words)

topic_list[0:6]

[['cocoa'],
 [],
 [],
 [],
 ['grain', 'wheat', 'corn', 'barley', 'oat', 'sorghum'],
 ['veg-oil',
  'linseed',
  'lin-oil',
  'soy-oil',
  'sun-oil',
  'soybean',
  'oilseed',
  'corn',
  'sunseed',
  'grain',
  'sorghum',
  'wheat']]

In [7]:
def pull_out_earn_topic(topic_list):
    for i, topic in enumerate(topic_list):
        
        # format is astrings, so this loop removes topics from nested list
        article_topics = ''
        for word in topic:
            article_topics += (word + ' ')
            
        # assign desired topic 
        if not article_topics:
            topic_list[i] = 'blank'
        elif 'earn' in article_topics:
            topic_list[i] = 'earn'
        else:
            topic_list[i] = 'other'
    
    return topic_list

In [8]:
topics_for_df = pull_out_earn_topic(topic_list)
df=pd.DataFrame(topics_for_df, columns=['topic'])

df

,topic
0,other
1,blank
2,blank
3,blank
4,other
...,...
995,blank
996,blank
997,earn
998,other


In [9]:
# pull out everything with the "text" tag from the bs4 object
all_text = soup.find_all("text")

# instatiate empty list 
list_all_text = list()

# loop through the bs4 element
for text in all_text:
    
    # getting just the text from the element
    # stripping out the newline indicator
    working_text = text.get_text().replace("\n", " ")
    
    # removing extra spaces
    working_text = ' '.join(working_text.split())
    
    # appending to list
    list_all_text.append(working_text)


In [12]:
df['text'] = list_all_text
df

,topic,text
0,other,"BAHIA COCOA REVIEW SALVADOR, Feb 26 - Showers ..."
1,blank,STANDARD OIL <SRD> TO FORM FINANCIAL UNIT CLEV...
2,blank,TEXAS COMMERCE BANCSHARES <TCB> FILES PLAN HOU...
3,blank,TALKING POINT/BANKAMERICA <BAC> EQUITY OFFER b...
4,other,NATIONAL AVERAGE PRICES FOR FARMER-OWNED RESER...
...,...,...
995,blank,ASHTON-TATE <TATE> TO OFFER COMMON SHARES TORR...
996,blank,KEYCORP <KEY> REGISTERS SUBORDINATED NOTES ALB...
997,earn,<NATIONAL SEA PRODUCTS LTD> 4TH QTR NET HALIFA...
998,other,U.K. MONEY MARKET SHORTAGE FORECAST REVISED DO...


In [11]:
print(df.iloc[1].text)

STANDARD OIL <SRD> TO FORM FINANCIAL UNIT CLEVELAND, Feb 26 - Standard Oil Co and BP North America Inc said they plan to form a venture to manage the money market borrowing and investment activities of both companies. BP North America is a subsidiary of British Petroleum Co Plc <BP>, which also owns a 55 pct interest in Standard Oil. The venture will be called BP/Standard Financial Trading and will be operated by Standard Oil under the oversight of a joint management committee. Reuter
